# Solving for RO - membrane Paramaters

In this demonstration, we will build a simple RO flowsheet and determine the membrane water permeability and salt permeability using an example module performance data.

### Example RO membrane specifications sheet

Usually membrane manufacturers provide specifications on membrane modules and not the permeability coefficients directly

<p style="text-align: center"><img src="graphics/BW30.png" height="400">

## Part 1: Build a simple RO flowsheet

## 1.1 Import necessary libraries

In [ ]:
from pyomo.environ import (
    check_optimal_termination,
    ConcreteModel,
    Constraint,
    value,
    Var,
    NonNegativeReals,
    assert_optimal_termination,
    units as pyunits,
)

from idaes.core import FlowsheetBlock
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from watertap.unit_models.reverse_osmosis_1D import (
    ReverseOsmosis1D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
)
from watertap.core.util.model_diagnostics.infeasible import *
from watertap.core.util.initialization import *
from idaes.core.util.model_statistics import degrees_of_freedom
from watertap.core.solvers import get_solver

## 1.2 Simple RO flowsheet

<p style="text-align: center"><img src="graphics/RO_Stage.png" width="50%">

### 1.3 Define ConcreteModel, FlowsheetBlock, and Property Package

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    has_full_reporting=True,
)

### 1.4 Specify values for system variables

In [ ]:
# fix the 4 inlet state variables
# feed mass flowrate of TDS (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(0.035)
# feed mass flowrate of water (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.inlet.pressure[0].fix(50e5)  # feed pressure (Pa)
m.fs.RO.inlet.temperature[0].fix(298)  # feed temperature (K)

# fix 2 membrane properties
m.fs.RO.A_comp.fix(4.2e-12)  # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)  # membrane salt permeability coeff (m/s)

# fix 4 module specficiations
m.fs.RO.area.fix(50)  # membrane stage area (m^2)
m.fs.RO.width.fix(5)  # membrane stage width (m)
m.fs.RO.feed_side.channel_height.fix(1e-3)  # channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)  # spacer porosity in membrane stage (-)

# 1 outlet state variable
m.fs.RO.permeate.pressure[0].fix(101325)  # permeate pressure (Pa)

In [ ]:
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

### 1.5 Scale all variables

In [ ]:
# Set scaling factors for component mass flowrates.
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))

# Set scaling factor for membrane area.
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate scaling factors for all other variables.
calculate_scaling_factors(m)

### 1.6 Initialize the model

In [ ]:
m.fs.RO.initialize()

### 1.7 Setup a solver and run a simulation

In [ ]:
# Solve the RO Unit
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

In [ ]:
# m.fs.RO.report()

## Calculate the A value based on permeate flux

In [ ]:
# We can start by unfixing the water permeability coefficient
m.fs.RO.A_comp.unfix()
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
# And then we can define a fixed value for the permeate flowrate
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].fix(0.4)
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
# Initialize and solve the model given the new fixed value
m.fs.RO.initialize()
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

In [ ]:
# m.fs.RO.report()
m.fs.RO.A_comp.display()

#### If RO recovery is defined in the specifications sheet

In [ ]:
# Similarly, we can also unfix the water permeate flowrate and fix the recovery to 50%
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].unfix()
m.fs.RO.recovery_vol_phase[0.0, "Liq"].fix(0.5)

print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

In [ ]:
# m.fs.RO.report()
m.fs.RO.A_comp.display()

## Part 2: Estimating membrane properties given a RO spec sheet

Setup RO unit model to reflect spec sheet system

<p style="text-align: center"><img src="graphics/BW30.png" height="400">


In [ ]:
permeate_flow = (48 * pyunits.m**3 / pyunits.day) * (
    997.0 * pyunits.kg / pyunits.m**3
)  # Volumetric flowrate * density to get mass flowrate
recovery = 0.15
feed_flow = permeate_flow / recovery
feed_conc = 2000 * pyunits.mg / pyunits.kg

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis1D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    transformation_scheme="BACKWARD",
    transformation_method="dae.finite_difference",
    finite_elements=10,
    has_full_reporting=True,
)

In [ ]:
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

## Specify values for system variables

In [ ]:
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(feed_flow * feed_conc)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(feed_flow)
m.fs.RO.inlet.pressure[0].fix(15.5 * pyunits.bar)
m.fs.RO.inlet.temperature[0].fix(298.15)

m.fs.RO.area.fix(41)
m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(3.5e-8)

m.fs.RO.permeate.pressure[0].fix(101325)
m.fs.RO.feed_side.channel_height.fix(1e-3)
m.fs.RO.feed_side.spacer_porosity.fix(0.95)
m.fs.RO.length.fix(1.016)

print("DOF = ", degrees_of_freedom(m))
print("RO DOF = ", degrees_of_freedom(m.fs.RO))
assert_no_degrees_of_freedom(m)

## Scale all variables.

In [ ]:
set_scaling_factor(m.fs.RO.area, 1e-2)
set_scaling_factor(m.fs.RO.feed_side.area, 1e-2)
set_scaling_factor(m.fs.RO.width, 1e-2)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))

calculate_scaling_factors(m)

## Note: 
This salinity given in the spec sheet is low (2g/L). Some of the variables in the NaCl property model are scaled and constrained to limits more relevant to seawater concentrations. Sometimes adjusting the bounds on these variables is required so solve in these different conditions

In [ ]:
# Release constraints related to low concentrations
for item in [m.fs.RO.permeate_side, m.fs.RO.feed_side.properties_interface]:
    for idx, param in item.items():
        if idx[1] > 0:
            param.molality_phase_comp["Liq", "NaCl"].setlb(1e-5)
            param.pressure_osm_phase["Liq"].setlb(100)

In [ ]:
def solve(m, raise_on_failure=True):
    # ---solving---
    solver = get_solver()

    print("\n--------- SOLVING ---------\n")
    results = solver.solve(m)

    if check_optimal_termination(results):
        print("\n--------- OPTIMAL SOLVE!!! ---------\n")

        print(
            f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}'
        )
        print(
            f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}'
        )
        print(
            f'{"Porosity":<20s}{value(m.fs.RO.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO.feed_side.spacer_porosity)):<10s}'
        )
        print("\n")

        return results
    assert False

In [ ]:
m.fs.RO.initialize()
results = solve(m)

#### Solve for A

In [ ]:
# Unfix A variable
m.fs.RO.A_comp.unfix()

# Fix the permeate flow
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].fix(permeate_flow)

print("DOF = ", degrees_of_freedom(m))

In [ ]:
results = solve(m)

#### Solve for B

In [ ]:
# Unfix B variable
m.fs.RO.B_comp.unfix()

# Fix the salt rejection
m.fs.RO.rejection_phase_comp[0, "Liq", "NaCl"].fix(0.997)

print("DOF = ", degrees_of_freedom(m))

In [ ]:
results = solve(m, raise_on_failure=True)

In [ ]:
m.fs.RO.A_comp.display()
m.fs.RO.B_comp.display()

# Try it yourself

## Solve for Pressure Loss and Spacer Porosity

<p style="text-align: center"><img src="graphics/BW30_2.png" width="80%">

In [ ]:
feed_flow = (19 * pyunits.m**3 / pyunits.hr) * (
    997.0 * pyunits.kg / pyunits.m**3
)  # Volumetric flowrate * density to get mass flowrate
pressure_loss = -1 * pyunits.bar

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis1D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    transformation_scheme="BACKWARD",
    transformation_method="dae.finite_difference",
    finite_elements=10,
    has_full_reporting=True,
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(feed_flow * feed_conc)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(feed_flow)
m.fs.RO.inlet.pressure[0].fix(41 * pyunits.bar)
m.fs.RO.inlet.temperature[0].fix(298.15)

m.fs.RO.area.fix(41)
m.fs.RO.A_comp.fix(1.159034619685113e-11)
m.fs.RO.B_comp.fix(2.2629627582609926e-08)

m.fs.RO.permeate.pressure[0].fix(101325)
m.fs.RO.feed_side.channel_height.fix(1e-3)
# m.fs.RO.feed_side.spacer_porosity.fix(0.95)
m.fs.RO.length.fix(1.016)

m.fs.RO.deltaP.fix(pressure_loss)

print("DOF = ", degrees_of_freedom(m))
print("RO DOF = ", degrees_of_freedom(m.fs.RO))
assert_no_degrees_of_freedom(m)

## Now fix the new feed flow rate and pressure loss

In [ ]:
# Fix the new flow rate
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(feed_flow)
# Fix the pressure drop
m.fs.RO.deltaP.fix(pressure_loss)

print("\n")
print("DOF = ", degrees_of_freedom(m))

## We have too few degrees of Freedom. Unfix the spacer porosity variable

In [ ]:
# Release constraints related to low concentrations
for item in [m.fs.RO.permeate_side, m.fs.RO.feed_side.properties_interface]:
    for idx, param in item.items():
        if idx[1] > 0:
            param.molality_phase_comp["Liq", "NaCl"].setlb(1e-5)
            param.pressure_osm_phase["Liq"].setlb(100)

In [ ]:
set_scaling_factor(m.fs.RO.area, 1e-2)
set_scaling_factor(m.fs.RO.feed_side.area, 1e-2)
set_scaling_factor(m.fs.RO.width, 1e-2)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))

calculate_scaling_factors(m)
m.fs.RO.initialize()

In [ ]:
results = solve(m)